In [1]:
#importing libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the dataset
dataset = pd.read_csv('diabetes_dataset.csv')
X = dataset[['Glucose', 'INS', 'BMI', 'DPF', 'Age']].values
y = dataset['Outcome'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define triangular membership function with optimized parameters
def triangular_mf(x, a, b, c, ki, wi):
    a = (a + ki) - wi
    b = (b + ki) - wi
    c = (c + ki) - wi
    return max(min((x - a) / (b - a), (c - x) / (c - b)), 0)

def fitness(params):
    fuzzy_results_train = [evaluate_rules(features, params) for features in X_train]
    y_pred_train = [round(result) for result in fuzzy_results_train]
    
    acc = accuracy_score(y_train, y_pred_train)
    tn, fp, fn, tp = confusion_matrix(y_train, y_pred_train).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    return 1 - (acc * 0.33 + sensitivity * 0.33 + specificity * 0.34)

class HarrisHawkOptimization:
    def __init__(self, fitness_func, dim, n_hawks=30, n_iterations=100):
        self.fitness_func = fitness_func
        self.dim = dim
        self.n_hawks = n_hawks
        self.n_iterations = n_iterations
        self.hawks = np.random.uniform(0, 100, (self.n_hawks, self.dim))
        self.rabbit = None

    def _find_rabbit(self):
        self.rabbit = min(self.hawks, key=self.fitness_func)

    def _update_position(self, hawk):
        E1 = np.random.uniform(0, 1)
        E2 = np.random.uniform(0, 1)
        
        if E1 < 0.5:
            if E2 < 0.5:
                hawk = self.rabbit - np.random.uniform(0, 1) * abs(self.rabbit - hawk)
            else:
                hawk = self.rabbit + np.random.uniform(0, 1) * abs(self.rabbit - hawk)
        else:
            hawk = np.random.uniform(0, 100, self.dim)
        
        return hawk

    def optimize(self):
        for iteration in range(self.n_iterations):
            self._find_rabbit()
            for i in range(self.n_hawks):
                new_hawk = self._update_position(self.hawks[i])
                if self.fitness_func(new_hawk) < self.fitness_func(self.hawks[i]):
                    self.hawks[i] = new_hawk

# Fuzzy rule evaluation with optimized parameters
def evaluate_rules(features, params):
    Glucose, INS, BMI, DPF, Age = features
    ki, wi = params  # or ki = params[0], wi = params[1] if params is an array
    
    # Membership functions for each feature, adjusted based on ki and wi
    Ageyoung = triangular_mf(Age, 0, 30, 40, ki, wi)
    
    Glucoselow = triangular_mf(Glucose, 0, 85, 100, ki, wi)
    Glucosemedium = triangular_mf(Glucose, 85, 125, 160, ki, wi)
    Glucosehigh = triangular_mf(Glucose, 125, 180, 200, ki, wi)
    
    INSlow = triangular_mf(INS, 0, 100, 200, ki, wi)
    INSmedium = triangular_mf(INS, 100, 200, 400, ki, wi)
    INShigh = triangular_mf(INS, 200, 400, 600, ki, wi)
    
    BMIlow = triangular_mf(BMI, 0, 20, 30, ki, wi)
    BMImedium = triangular_mf(BMI, 20, 30, 40, ki, wi)
    BMIhigh = triangular_mf(BMI, 30, 40, 50, ki, wi)
    
    DPFlow = triangular_mf(DPF, 0, 0.3, 0.6, ki, wi)
    DPFmedium = triangular_mf(DPF, 0.3, 0.6, 1.0, ki, wi)
    DPFhigh = triangular_mf(DPF, 0.6, 1.0, 2.0, ki, wi)
    
    # Fuzzy rules
    DMverylow = max(Glucoselow, INSlow, BMIlow, DPFlow, Ageyoung)
    DMlow = max(Glucoselow, INSlow, BMIhigh, DPFlow, Ageyoung)
    DMmedium = max(Glucosemedium, INShigh, BMIhigh, DPFmedium, Ageyoung)
    DMhigh = max(Glucosehigh, INSmedium, BMIhigh, DPFhigh, Ageyoung)
    DMverylow = max(Glucoselow, INSlow, BMImedium, DPFlow, Ageyoung)

    # Aggregate into a single value for DM
    DM = max(DMverylow, DMlow, DMmedium, DMhigh)
    
    return DM

# Initialize HHO
hho = HarrisHawkOptimization(fitness, dim=2, n_hawks=30, n_iterations=100)  # Here dim=2 for ki and wi

# Run Optimization
hho.optimize()

# Get the best hawk (solution)
best_hawk = min(hho.hawks, key=fitness)
print("Best Parameters ki and wi: ", best_hawk)

# Test the classifier on the test set using optimized parameters
fuzzy_results_test = [evaluate_rules(features, best_hawk) for features in X_test]
y_pred_test = [round(result) for result in fuzzy_results_test]

# Calculate performance metrics
acc = accuracy_score(y_test, y_pred_test)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"Accuracy based on optimized parameters: {acc}")
print(f"Sensitivity based on optimized parameters: {sensitivity}")
print(f"Specificity based on optimized parameters: {specificity}")


/home/el-sunais/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Best Parameters ki and wi:  [ 35.86622213 -66.4976074 ]
Accuracy based on optimized parameters: 0.7142857142857143
Sensitivity based on optimized parameters: 0.6181818181818182
Specificity based on optimized parameters: 0.7676767676767676
